In [156]:
import pandas as pd
import sqlite3
import numpy as np
import geopandas as gpd
from shapely.geometry import Point

from shapely.geometry import Point
con = sqlite3.connect("natural_disasters.db")
df = pd.read_sql_query('SELECT * FROM natural_disasters_data', con)
df.dropna(axis=1, inplace=True)
df.head()
df.columns

Index(['Year', 'Seq', 'Glide', 'Disaster_Group', 'Disaster_Subgroup',
       'Disaster_Type', 'Disaster_Subtype', 'Country', 'ISO', 'Region',
       'Continent', 'Origin', 'Associated_Dis', 'Associated_Dis2',
       'Aid_Contribution', 'Dis_Mag_Value', 'Dis_Mag_Scale', 'Latitude',
       'Longitude', 'Local_Time', 'Start_Year', 'Start_Month', 'Start_Day',
       'End_Year', 'End_Month', 'End_Day', 'Total_Deaths', 'No_Injured',
       'No_Affected', 'No_Homeless', 'Total_Affected', 'Insured_Damages_USD',
       'Total_Damages_USD', 'CPI'],
      dtype='object')

In [157]:
df_1 = df[['Year','Country','Disaster_Type','Continent','Latitude','Longitude','Total_Deaths','Seq']]
df_1.head()

,Year,Country,Disaster_Type,Continent,Latitude,Longitude,Total_Deaths,Seq
0,2020,Angola,Flood,Africa,-913.908,142.007,30,173
1,2016,Angola,Flood,Africa,-832.104,13.980,23,131
2,2020,Peru,Flood,Americas,-701.073,-768.801,,122
3,2016,"Tanzania, United Republic of",Flood,Africa,-689.427,360.000,13,244
4,2016,"Tanzania, United Republic of",Flood,Africa,-476.168,355.050,3,28


In [158]:
df_1['Latitude'].astype('Float64')
df_1['Longitude'].astype('Float64')



0        142.007
1          13.98
2       -768.801
3          360.0
4         355.05
          ...   
16113        0.0
16114        0.0
16115        0.0
16116        0.0
16117        0.0
Name: Longitude, Length: 16118, dtype: Float64

In [159]:

df_1 = df_1[~((df['Latitude'] == 0.000) | (df['Longitude'] == 0.000))]
df_1

,Year,Country,Disaster_Type,Continent,Latitude,Longitude,Total_Deaths,Seq
0,2020,Angola,Flood,Africa,-913.908,142.007,30,173
1,2016,Angola,Flood,Africa,-832.104,13.980,23,131
2,2020,Peru,Flood,Americas,-701.073,-768.801,,122
3,2016,"Tanzania, United Republic of",Flood,Africa,-689.427,360.000,13,244
4,2016,"Tanzania, United Republic of",Flood,Africa,-476.168,355.050,3,28
...,...,...,...,...,...,...,...,...
2717,1991,Costa Rica,Earthquake,Americas,9.685,-83.073,47,114
2718,1991,Panama,Earthquake,Americas,9.685,-83.073,30,114
2719,1990,Costa Rica,Earthquake,Americas,9.869,84.302,1,170
2720,1990,Indonesia,Earthquake,Asia,-9.904,101.120,,702


In [160]:
df_1['Total_Deaths'] = df_1['Total_Deaths'].fillna(np.nan)
df_1['Total_Deaths']= df_1['Total_Deaths'].replace('','0')
df_1['Total_Deaths'] = df_1['Total_Deaths'].astype('int32')

In [161]:
df_1

,Year,Country,Disaster_Type,Continent,Latitude,Longitude,Total_Deaths,Seq
0,2020,Angola,Flood,Africa,-913.908,142.007,30,173
1,2016,Angola,Flood,Africa,-832.104,13.980,23,131
2,2020,Peru,Flood,Americas,-701.073,-768.801,0,122
3,2016,"Tanzania, United Republic of",Flood,Africa,-689.427,360.000,13,244
4,2016,"Tanzania, United Republic of",Flood,Africa,-476.168,355.050,3,28
...,...,...,...,...,...,...,...,...
2717,1991,Costa Rica,Earthquake,Americas,9.685,-83.073,47,114
2718,1991,Panama,Earthquake,Americas,9.685,-83.073,30,114
2719,1990,Costa Rica,Earthquake,Americas,9.869,84.302,1,170
2720,1990,Indonesia,Earthquake,Asia,-9.904,101.120,0,702


In [162]:
total_death = df_1.groupby(['Continent','Country'])['Total_Deaths'].sum()
total_death

Continent  Country          
Africa     Algeria              6968
           Angola                353
           Botswana                0
           Burkina Faso           68
           Burundi               104
                                ... 
Oceania    Samoa                 148
           Solomon Islands       433
           Tonga                  10
           Vanuatu                12
           Wallis and Futuna       5
Name: Total_Deaths, Length: 167, dtype: int32

In [163]:
Frequency = df_1.groupby(['Continent','Country'])['Seq'].sum()
Frequency

Continent  Country          
Africa     Algeria              6270
           Angola               1799
           Botswana              233
           Burkina Faso         1219
           Burundi              2106
                                ... 
Oceania    Samoa                 420
           Solomon Islands      1331
           Tonga                 498
           Vanuatu              3713
           Wallis and Futuna     134
Name: Seq, Length: 167, dtype: int64

In [164]:
Geo_map_data= df_1.groupby(['Country','Disaster_Type','Continent','Latitude','Longitude','Total_Deaths','Seq'])['Seq'].sum()
Geo_map_data


Country      Disaster_Type  Continent  Latitude  Longitude  Total_Deaths  Seq
Afghanistan  Earthquake     Asia        33.377   71.317     14            202    202
                                        33.426   69.524     2             436    436
                                        33.825   59.809     5             95      95
                                        34.010   71.400     12            166    166
                                        34.185   70.076     22            156    156
                                                                                ... 
Zambia       Flood          Africa     -12.950   24.370     4             10      10
                                       -10.620   32.590     0             40      40
Zimbabwe     Flood          Africa     -20.780   31.240     24            616    616
                                       -19.041   30.804     26            94      94
                                       -16.660   31.240     3           

In [165]:
Geo_map_data = df_1.groupby(['Country','Disaster_Type','Continent','Latitude','Longitude']).agg(Total_Deaths=('Total_Deaths', 'sum'), Seq=('Seq', 'sum')).reset_index()

In [166]:
Geo_map_data_json = Geo_map_data.to_json(orient="records")

with open('Geo_map_data.json', 'w') as json_file:
    json_file.write(Geo_map_data_json)

In [171]:
#Geo_map_data_deathCount = df_1.groupby(['Country','Disaster_Type','Continent','Latitude','Longitude']).agg(Total_Deaths=('Total_Deaths', 'sum'), Seq=('Seq', 'sum')).reset_index()
Geo_map_data_deathCount= df_1.groupby(['Continent','Country'])['Total_Deaths'].count()
Geo_map_data2



Continent  Country          
Africa     Algeria              29
           Angola               14
           Botswana              1
           Burkina Faso          4
           Burundi               8
                                ..
Oceania    Samoa                 1
           Solomon Islands      10
           Tonga                 2
           Vanuatu              11
           Wallis and Futuna     1
Name: Total_Deaths, Length: 167, dtype: int64

In [177]:
total_deaths_by_country = df_1.groupby(['Country','Disaster_Type','Continent','Latitude','Longitude']).agg(Total_Deaths=('Total_Deaths', 'sum'), Seq=('Seq', 'sum')).reset_index()
death_count_by_country = df_1.groupby(['Continent','Country'])['Total_Deaths'].count().reset_index()

In [180]:
death_count_by_country

,Continent,Country,Total_Deaths
0,Africa,Algeria,29
1,Africa,Angola,14
2,Africa,Botswana,1
3,Africa,Burkina Faso,4
4,Africa,Burundi,8
...,...,...,...
162,Oceania,Samoa,1
163,Oceania,Solomon Islands,10
164,Oceania,Tonga,2
165,Oceania,Vanuatu,11


In [181]:
total_deaths_by_country

,Country,Disaster_Type,Continent,Latitude,Longitude,Total_Deaths,Seq
0,Afghanistan,Earthquake,Asia,33.377,71.317,14,202
1,Afghanistan,Earthquake,Asia,33.426,69.524,2,436
2,Afghanistan,Earthquake,Asia,33.825,59.809,5,95
3,Afghanistan,Earthquake,Asia,34.010,71.400,12,166
4,Afghanistan,Earthquake,Asia,34.185,70.076,22,156
...,...,...,...,...,...,...,...
2663,Zambia,Flood,Africa,-12.950,24.370,4,10
2664,Zambia,Flood,Africa,-10.620,32.590,0,40
2665,Zimbabwe,Flood,Africa,-20.780,31.240,24,616
2666,Zimbabwe,Flood,Africa,-19.041,30.804,26,94


In [183]:
total_deaths_by_country = df_1.groupby(['Country', 'Disaster_Type', 'Continent', 'Latitude', 'Longitude']).agg(Total_Deaths=('Total_Deaths', 'sum')).reset_index()

print(total_deaths_by_country.head())

       Country Disaster_Type Continent  Latitude  Longitude  Total_Deaths
0  Afghanistan    Earthquake      Asia    33.377     71.317            14
1  Afghanistan    Earthquake      Asia    33.426     69.524             2
2  Afghanistan    Earthquake      Asia    33.825     59.809             5
3  Afghanistan    Earthquake      Asia    34.010     71.400            12
4  Afghanistan    Earthquake      Asia    34.185     70.076            22


In [184]:
total_deaths_by_country.to_json("total_deaths_by_country.json", orient="records", lines=True)